This script is for sharing data with client externally. 

Data will NOT be trimmed to 480-1050nm, 3nm resolution.

In [26]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.offline
from plotly.subplots import make_subplots
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler

from pptoolbox.platform import preprocess, WAVELENGTHS_3NM_V1
from pptoolbox.preprocessing import SNV, MSC, SavitzkyGolay, Detrend
# from pptoolbox.visualization import (plot_spectra, plot_loadings, plot_scree, fit_pca,
#                                                generate_pc_df, plot_svp, initialize_theme)

from pptoolbox.platform.preprocessing import unpack_json

# import data

In [17]:
from pathlib import Path
datafolder_path = Path ("C:/Users/RyanSoh/OneDrive - ProfilePrint/Projects/Mars NA")
raw_csv = datafolder_path / "raw" / "mars_raw_data.csv"

raw_df = pd.read_csv(raw_csv).set_index('specimen_id')
raw_df.head()

,lot_id,lot_name,date_scanned,analyser_id,company_id,product_id,product_name,raw_data,dark_ref_data,white_ref_data,dark_ref_scan_time,white_ref_scan_time
specimen_id,,,,,,,,,,,,
470043,78616,Cocoa Nibs - 139,1727383214,86,1291,6698,Cocoa Nibs,"[{""wavelength"": 337.72271728515625, ""reflectan...","[149.34652542372882, 156.51101694915255, 152.4...","[149.56911016949152, 156.63889830508475, 152.4...",1727382840,1727382860
470044,78616,Cocoa Nibs - 139,1727383235,86,1291,6698,Cocoa Nibs,"[{""wavelength"": 337.72271728515625, ""reflectan...","[149.34652542372882, 156.51101694915255, 152.4...","[149.56911016949152, 156.63889830508475, 152.4...",1727382840,1727382860
470045,78616,Cocoa Nibs - 139,1727383254,86,1291,6698,Cocoa Nibs,"[{""wavelength"": 337.72271728515625, ""reflectan...","[149.34652542372882, 156.51101694915255, 152.4...","[149.56911016949152, 156.63889830508475, 152.4...",1727382840,1727382860
470048,78616,Cocoa Nibs - 139,1727383282,86,1291,6698,Cocoa Nibs,"[{""wavelength"": 337.72271728515625, ""reflectan...","[149.34652542372882, 156.51101694915255, 152.4...","[149.56911016949152, 156.63889830508475, 152.4...",1727382840,1727382860
470051,78616,Cocoa Nibs - 139,1727383315,86,1291,6698,Cocoa Nibs,"[{""wavelength"": 337.72271728515625, ""reflectan...","[149.34652542372882, 156.51101694915255, 152.4...","[149.56911016949152, 156.63889830508475, 152.4...",1727382840,1727382860


# unpack json

In [18]:
spectra_df = abs(unpack_json(raw_df))

spectra_df

,337.722717,338.319275,338.915771,339.512268,340.108765,340.705200,341.301605,341.898010,342.494385,343.090729,...,1471.369751,1471.869751,1472.369873,1472.869873,1473.369873,1473.869629,1474.369385,1474.869019,1475.368774,1475.868164
specimen_id,,,,,,,,,,,,,,,,,,,,,
470043,10.812869,113.353214,680.487805,373.485303,68.970814,108.739985,4.820453,57.824639,205.725699,398.923187,...,13.085588,11.720125,8.401503,20.829793,24.041106,12.173211,21.595933,19.839181,40.362469,61.401175
470044,55.834761,80.583168,989.593496,161.367726,75.921659,38.674436,6.225605,137.180910,165.845539,213.065327,...,24.413096,25.077818,34.335618,16.142736,20.466059,50.368426,37.201592,21.293860,28.251841,71.233788
470045,20.064725,173.260437,1530.243902,403.959208,225.614439,546.249090,43.208431,235.516093,508.921438,371.284996,...,8.032424,3.139802,21.910237,32.625319,33.637285,6.884021,18.832891,11.593567,7.003965,40.483317
470048,373.005901,460.834990,2236.747967,765.446911,524.039939,1157.319738,315.241998,2041.398446,1031.890812,1056.999282,...,209.211496,257.856273,214.127791,258.156330,296.077580,194.196023,215.848806,137.456140,197.007740,253.009644
470051,312.392918,391.252485,2509.268293,831.553689,606.413210,1071.158048,361.748634,1597.780244,886.551265,961.809045,...,222.107590,271.159832,213.696662,239.323138,235.620206,231.270819,259.129089,170.519006,187.124788,254.173595
470056,267.542357,328.164347,1216.097561,618.116377,416.513057,885.724690,267.642467,1137.402886,524.500666,582.770998,...,169.786293,217.742590,164.105682,188.530161,195.484151,134.611891,155.050840,96.337719,151.151595,190.655138
470059,394.403198,540.954274,2270.894309,933.413317,614.516129,1272.469046,413.836846,1805.660377,1072.836218,1263.460158,...,239.077798,303.248071,239.188592,279.750779,256.723115,197.234279,250.055261,154.071637,206.890693,271.965414
470060,116.295450,4.208085,112.032520,24.415117,14.016897,125.054625,97.697112,359.822420,40.412783,99.928212,...,54.732077,119.664366,58.854742,71.141320,70.487770,73.231049,96.640141,62.288012,63.535964,101.607361
470061,308.014468,442.312790,1026.666667,835.632873,423.617512,1064.311726,305.932865,1536.293008,949.134487,895.190237,...,206.558585,256.787116,178.797258,261.993769,212.244898,178.580801,205.758179,150.745614,190.135926,240.050992


# scan info

In [19]:
# can drop columns be it for whether y labels are from db
COLS_TO_DROP = [
    'raw_data',
    'dark_ref_data',
    'white_ref_data',
    'dark_ref_scan_time',
    'white_ref_scan_time',
    # THIS IS FOR SHARING DATA EXTERNALLY, DROP IRRELEVANT COLS
    'lot_id',
    'company_id',
    'product_id',
    'analyser_id'
    # 'integration_time',
    # 'detector_temp',
    # 'spectrometer_temp'
]
info_df = \
    raw_df \
    .drop(
        columns= COLS_TO_DROP
    )

info_df

,lot_name,date_scanned,product_name
specimen_id,,,
470043,Cocoa Nibs - 139,1727383214,Cocoa Nibs
470044,Cocoa Nibs - 139,1727383235,Cocoa Nibs
470045,Cocoa Nibs - 139,1727383254,Cocoa Nibs
470048,Cocoa Nibs - 139,1727383282,Cocoa Nibs
470051,Cocoa Nibs - 139,1727383315,Cocoa Nibs
470056,Cocoa Nibs - 139,1727383387,Cocoa Nibs
470059,Cocoa Nibs - 139,1727383413,Cocoa Nibs
470060,Cocoa Nibs - 139,1727383438,Cocoa Nibs
470061,Cocoa Nibs - 139,1727383496,Cocoa Nibs


In [29]:
info_df['date_scanned'] = pd.to_datetime(info_df["date_scanned"], unit = 's').dt.date
info_df

,lot_name,date_scanned,product_name
specimen_id,,,
470043,Cocoa Nibs - 139,2024-09-26,Cocoa Nibs
470044,Cocoa Nibs - 139,2024-09-26,Cocoa Nibs
470045,Cocoa Nibs - 139,2024-09-26,Cocoa Nibs
470048,Cocoa Nibs - 139,2024-09-26,Cocoa Nibs
470051,Cocoa Nibs - 139,2024-09-26,Cocoa Nibs
470056,Cocoa Nibs - 139,2024-09-26,Cocoa Nibs
470059,Cocoa Nibs - 139,2024-09-26,Cocoa Nibs
470060,Cocoa Nibs - 139,2024-09-26,Cocoa Nibs
470061,Cocoa Nibs - 139,2024-09-26,Cocoa Nibs


# merge scans and info

In [30]:
joined_df = \
    pd.merge(
        info_df,
        spectra_df.reset_index(),
        on = 'specimen_id',
        how = 'left'
    ) \
    .set_index('specimen_id')

raw_csv = datafolder_path / "raw" / "mars_raw_data_tech_assessment_shared.csv"
joined_df.to_csv(raw_csv)

joined_df

,lot_name,date_scanned,product_name,337.72271728515625,338.31927490234375,338.915771484375,339.51226806640625,340.1087646484375,340.7052001953125,341.3016052246094,...,1471.3697509765625,1471.8697509765625,1472.369873046875,1472.869873046875,1473.369873046875,1473.86962890625,1474.369384765625,1474.8690185546875,1475.3687744140625,1475.8681640625
specimen_id,,,,,,,,,,,,,,,,,,,,,
470043,Cocoa Nibs - 139,2024-09-26,Cocoa Nibs,10.812869,113.353214,680.487805,373.485303,68.970814,108.739985,4.820453,...,13.085588,11.720125,8.401503,20.829793,24.041106,12.173211,21.595933,19.839181,40.362469,61.401175
470044,Cocoa Nibs - 139,2024-09-26,Cocoa Nibs,55.834761,80.583168,989.593496,161.367726,75.921659,38.674436,6.225605,...,24.413096,25.077818,34.335618,16.142736,20.466059,50.368426,37.201592,21.293860,28.251841,71.233788
470045,Cocoa Nibs - 139,2024-09-26,Cocoa Nibs,20.064725,173.260437,1530.243902,403.959208,225.614439,546.249090,43.208431,...,8.032424,3.139802,21.910237,32.625319,33.637285,6.884021,18.832891,11.593567,7.003965,40.483317
470048,Cocoa Nibs - 139,2024-09-26,Cocoa Nibs,373.005901,460.834990,2236.747967,765.446911,524.039939,1157.319738,315.241998,...,209.211496,257.856273,214.127791,258.156330,296.077580,194.196023,215.848806,137.456140,197.007740,253.009644
470051,Cocoa Nibs - 139,2024-09-26,Cocoa Nibs,312.392918,391.252485,2509.268293,831.553689,606.413210,1071.158048,361.748634,...,222.107590,271.159832,213.696662,239.323138,235.620206,231.270819,259.129089,170.519006,187.124788,254.173595
470056,Cocoa Nibs - 139,2024-09-26,Cocoa Nibs,267.542357,328.164347,1216.097561,618.116377,416.513057,885.724690,267.642467,...,169.786293,217.742590,164.105682,188.530161,195.484151,134.611891,155.050840,96.337719,151.151595,190.655138
470059,Cocoa Nibs - 139,2024-09-26,Cocoa Nibs,394.403198,540.954274,2270.894309,933.413317,614.516129,1272.469046,413.836846,...,239.077798,303.248071,239.188592,279.750779,256.723115,197.234279,250.055261,154.071637,206.890693,271.965414
470060,Cocoa Nibs - 139,2024-09-26,Cocoa Nibs,116.295450,4.208085,112.032520,24.415117,14.016897,125.054625,97.697112,...,54.732077,119.664366,58.854742,71.141320,70.487770,73.231049,96.640141,62.288012,63.535964,101.607361
470061,Cocoa Nibs - 139,2024-09-26,Cocoa Nibs,308.014468,442.312790,1026.666667,835.632873,423.617512,1064.311726,305.932865,...,206.558585,256.787116,178.797258,261.993769,212.244898,178.580801,205.758179,150.745614,190.135926,240.050992
